# 🔍 Chapter 7: End-to-End Feature Extraction

Feature extraction is the bridge between raw coordinates (X, Y, Z) and semantic understanding. By analyzing the geometric neighborhood of each point, we can determine if it belongs to a wall (planar), a pole (linear), or vegetation (scattered).

**Objectives:**
1.  **Neighborhood Concepts**: Understanding K-NN and Radius Search.
2.  **Geometric Features**: Calculating Planarity, Linearity, Verticality, and Omnivariance.
3.  **Visualization**: Mapping these features to colors for visual analysis.

In [ ]:
import numpy as np
import pyvista as pv
import time
from scipy.spatial import KDTree

# Set plot theme
pv.set_plot_theme("document")

## 1. Load Data with PyVista

PyVista is a powerful wrapper around VTK, great for mesh analysis and rendering.

In [ ]:
file_path = "../DATA/features_verviers.ply"

try:
    pcd = pv.read(file_path)
    print(f"Loaded {pcd.n_points} points.")
    
    # Quick visualization with Eye Dome Lighting (EDL) for depth perception
    pcd.plot(eye_dome_lighting=True, point_size=3)
except FileNotFoundError:
    print(f"⚠️ Error: {file_path} not found.")

## 2. Neighborhood Search (KD-Tree)

To analyze the geometry around a point, we first need to find its neighbors. We use `scipy.spatial.KDTree` for efficient querying.

In [ ]:
# Build Tree
start = time.time()
tree = KDTree(pcd.points)
print(f"KD-Tree built in {time.time() - start:.2f} seconds.")

# Query K-Nearest Neighbors for ALL points
k = 20
dists, indices = tree.query(pcd.points, k=k)
print(f"Found {k} neighbors for all points.")

## 3. Geometric Feature Extraction (PCA)

For each neighborhood, we compute the covariance matrix and its eigenvalues ($λ_1, λ_2, λ_3$). These describe the shape of the neighborhood.

*   **Linearity**: $(λ_1 - λ_2) / λ_1$ (Lines like poles, wires)
*   **Planarity**: $(λ_2 - λ_3) / λ_1$ (Surfaces like ground, walls)
*   **Sphericity/Scattering**: $λ_3 / λ_1$ (Vegetation)
*   **Verticality**: $1 - |N_z|$ (How vertical is the normal vector?)

In [ ]:
def compute_features(points_subset):
    # PCA
    mean = np.mean(points_subset, axis=0)
    centered = points_subset - mean
    cov = np.cov(centered, rowvar=False)
    eigenvalues, eigenvectors = np.linalg.eig(cov)
    
    # Sort eigenvalues
    sort_idx = np.argsort(eigenvalues)[::-1]
    vals = eigenvalues[sort_idx]
    vecs = eigenvectors[:, sort_idx]
    
    l1, l2, l3 = vals[0], vals[1], vals[2]
    
    # Avoid division by zero
    if l1 == 0:
        return 0, 0, 0, 0

    linearity = (l1 - l2) / l1
    planarity = (l2 - l3) / l1
    sphericity = l3 / l1
    
    # Verticality (using Normal vector which is the 3rd eigenvector)
    normal = vecs[:, 2]
    verticality = 1 - abs(normal[2])
    
    return linearity, planarity, sphericity, verticality

# Test on one point
subset = pcd.points[indices[0]]
lin, plan, sph, vert = compute_features(subset)
print(f"Sample Point Features -> Lin: {lin:.2f}, Plan: {plan:.2f}, Sph: {sph:.2f}, Vert: {vert:.2f}")

## 4. Processing the Full Cloud

We loop through all points (or a subset for speed) to compute these features.

In [ ]:
# Let's process a subset for demonstration speed (first 5000 points)
# In production, you would vectorize this or run on all points
n_process = 5000

linearity_arr = np.zeros(n_process)
planarity_arr = np.zeros(n_process)
verticality_arr = np.zeros(n_process)

print(f"Processing first {n_process} points...")
start = time.time()
for i in range(n_process):
    subset = pcd.points[indices[i]]
    l, p, s, v = compute_features(subset)
    linearity_arr[i] = l
    planarity_arr[i] = p
    verticality_arr[i] = v
print(f"Done in {time.time() - start:.2f} seconds.")

# Assign to Cloud (Only the processed part)
pcd_sub = pcd.extract_points(range(n_process))
pcd_sub['Linearity'] = linearity_arr
pcd_sub['Planarity'] = planarity_arr
pcd_sub['Verticality'] = verticality_arr

## 5. Visualizing Features

Visualize Planarity (Walls/Ground should shine) and Verticality (Facades).

In [ ]:
pcd_sub.plot(scalars='Planarity', cmap='magma', point_size=5, title="Planarity")
pcd_sub.plot(scalars='Verticality', cmap='viridis', point_size=5, title="Verticality")